<a href="https://colab.research.google.com/github/Streit-C/MSE6230/blob/main/MonteCarlo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preamble: Run the cells below to import the necessary Python packages. Answer each question below in a separate document (Word, Google Docs, etc.) for submission. Make sure to copy the images you generate into this document.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Part 1: Introduction to Monte Carlo Simulations via Random Walks

This section introduces the fundamental concepts of Monte Carlo (MC) simulations through the example of a random walk. You will run and analyze provided Python code to build intuition about stochastic processes and statistical properties that underpin more complex MC methods used in statistical mechanics.

A random walk is a mathematical model describing a path consisting of a sequence of random steps. In a 1D random walk, a particle moves either one step to the right (+1) or one step to the left (−1) with equal probability at each time step. Over many steps, the particle’s position evolves randomly, and meaningful statistical properties can be analyzed.

---


Let's plot 10 1D random walks of 1000 steps each.

In [ ]:
def random_walk_1d(n_steps):
    steps = np.random.choice([-1, 1], size=n_steps)
    position = np.cumsum(steps)
    return position

plt.figure(figsize=(10,6))
n_steps = 1000

for _ in range(10):
    pos = random_walk_1d(1000)
    plt.plot(pos, alpha=0.6)
plt.title("Ten 1D Random Walks (1000 steps each)")
plt.xlabel("Step number")
plt.ylabel("Position")
plt.grid(True)
plt.show()

**Question 1**: Describe the main features of the random walk trajectories. What do you notice about the spread of the paths as the number of steps increases?

---

Now, let's look at the distribution of final positions for 10,000 different random walks.

In [ ]:
n_walks = 10000
final_positions = np.array([random_walk_1d(n_steps)[-1] for _ in range(n_walks)])

plt.figure(figsize=(8,5))
plt.hist(final_positions, bins=50, density=True, alpha=0.7, color='c')
plt.title("Distribution of Final Positions After 1000 Steps")
plt.xlabel("Final Position")
plt.ylabel("Probability Density")
plt.grid(True)
plt.show()

**Question 2**: What is the shape of the distribution of final positions? How does this relate to the Central Limit Theorem?

---

Plot the mean squared displacement (MSD), defined as:

$MSD(n)=⟨(x_n−x_0)^2⟩$

as a function of the number of steps n. Here, $x_0=0$ is the starting position.

In [ ]:
max_steps = 1000
msd = np.zeros(max_steps)
for step in range(1, max_steps+1):
    positions = np.array([random_walk_1d(step)[-1] for _ in range(1000)])
    msd[step-1] = np.mean(positions**2)

plt.figure(figsize=(8,5))
plt.plot(range(1, max_steps+1), msd, label="Simulated MSD")
plt.plot(range(1, max_steps+1), range(1, max_steps+1), 'r--', label="Theoretical MSD (linear)")
plt.title("Mean Squared Displacement vs Number of Steps")
plt.xlabel("Number of Steps")
plt.ylabel("Mean Squared Displacement")
plt.legend()
plt.grid(True)
plt.show()

**Question 3**: How does the MSD grow as the number of steps increases? Is this consistent with theoretical expectations for a random walk?

# Part 2: The Metropolis Algorithm and Grain Boundary Stability

This section introduces the Metropolis Monte Carlo algorithm and its application to modeling grain boundary stability in metal alloys. You will use provided Python code to simulate a simplified 2D lattice model, analyze how grain boundaries evolve, and interpret the results in the context of thermodynamics and materials science.

Grain boundaries are interfaces between different crystalline orientations in polycrystalline metals. Their stability affects material properties such as mechanical response. In this section, we use a lattice model (similar to the Ising model) to represent grains: each site on a 2D grid has a "grain orientation" (an integer label). The "energy" of the system increases when neighboring sites have different orientations, analogous to the energy cost of grain boundaries.

The Metropolis algorithm is a Monte Carlo method for simulating the equilibrium properties of such systems. It works by proposing random changes to the system and accepting or rejecting them based on the change in energy and the system temperature, allowing the model to explore states according to the Boltzmann distribution. A basic outline is as follows:



*   Randomly select a site.
*   Propose changing its orientation.
*   Calculate the change in system energy ($ΔE$).
*   Accept or reject the change based on the Metropolis criterion:
  
   $P(accept)=
   \begin{cases}
    1 \text{,} \ \text{if} \ ΔE≤0 \\
    \text{exp}⁡(−ΔE/kT) \text{,} \ \text{if} \ ΔE>0
   \end{cases}
   $
*   Repeat for many steps to observe evolution toward equilibrium.


---

Let's start by generating a random seed tied to your UVA computing ID. Change seed_string below to be your UVA computing ID. Note that randomizer seeds will advance after being used, so if you want to re-run any of the code, restart from the section where your randomizer seed is generated.

In [ ]:
def string_to_seed(s, default_value="UVA"):
    """Convert a string to a reproducible integer seed."""
    if s == default_value:
        raise ValueError(
            f"Error: seed string is set to the default value '{default_value}'. "
            "Please change it to your unique identifier (e.g., your computing ID)."
        )
    return abs(hash(s)) % (2**32)

seed_string = "UVA"  # Replace with your computing ID
seed = string_to_seed(seed_string)
rng = np.random.default_rng(seed)

---

Generate your initial lattice below. Note the initial lattice size (L = 50) and the number of unique grain orientations (n = 5).

In [ ]:
# Parameters
L = 50  # Lattice size (LxL)
n_orient = 5  # Number of grain orientations
k_B = 1.380649e-23  # Boltzmann constant in J/K

# Initialize lattice with random orientations
lattice = rng.integers(0, n_orient, size=(L, L))

def plot_lattice(lattice, title=""):
    plt.figure(figsize=(6,6))
    plt.imshow(lattice, cmap='tab20', interpolation='nearest')
    plt.title(title)
    plt.axis('off')
    plt.show()

plot_lattice(lattice, title="Initial Grain Orientation Configuration")


---

Run the code below to perform the Monte Carlo simulation using the Metropolis algorithm and visualize the evolution. This code will likely take over a minute to finish.

In [ ]:
def energy_site(lattice, i, j):
    # Periodic boundary conditions
    L = lattice.shape[0]
    s = lattice[i, j]
    neighbors = [
        lattice[(i+1)%L, j],
        lattice[(i-1)%L, j],
        lattice[i, (j+1)%L],
        lattice[i, (j-1)%L]
    ]
    # Energy: +1 for each unlike neighbor
    return sum([1 for n in neighbors if n != s])

def metropolis_step(lattice, T, rng):
    L = lattice.shape[0]
    for _ in range(L*L):
        i, j = rng.integers(0, L, size=2)
        current = lattice[i, j]
        proposal = rng.choice([x for x in range(n_orient) if x != current])
        dE = 0
        for di, dj in [(-1,0),(1,0),(0,-1),(0,1)]:
            ni, nj = (i+di)%L, (j+dj)%L
            if lattice[ni, nj] != current:
                dE -= 1
            if lattice[ni, nj] != proposal:
                dE += 1
        dE_joules = dE * E_0
        if dE <= 0 or rng.random() < np.exp(-dE_joules / (k_B * T)):
            lattice[i, j] = proposal
    return lattice

def grain_boundary_fraction(lattice):
    L = lattice.shape[0]
    gb_sites = 0
    for i in range(L):
        for j in range(L):
            s = lattice[i, j]
            for di, dj in [(-1,0),(1,0),(0,-1),(0,1)]:
                ni, nj = (i+di)%L, (j+dj)%L
                if lattice[ni, nj] != s:
                    gb_sites += 1
    # Each boundary counted twice (once from each side)
    return gb_sites / (4*L*L)

# Simulation parameters
T = 300.0  # "Temperature in Kelvin"
n_steps = 1000
E_0 = 5e-20  # Joules per site (This is included because I want to use actual values for T.
             # If this weren't included, the boltzman constant dominates and temperature changes are meaningless.)

# Run simulation and visualize
lattice_sim = lattice.copy()
gb_fractions = []

for step in range(n_steps):
    lattice_sim = metropolis_step(lattice_sim, T, rng)
    gb_fractions.append(grain_boundary_fraction(lattice_sim))
    if step in [0, 1, 9, 49, 99, 499, 999]:
        plot_lattice(lattice_sim, title=f"After {step+1} MC Steps")

**Question 4**: How does the grain structure change as the simulation progresses? What is happening to the grain boundaries?


---

Now, let's plot the fraction of sites that are on a grain boundary as a function of Monte Carlo steps.

In [ ]:
# Plot grain boundary fraction
plt.figure(figsize=(7,5))
plt.plot(range(1, n_steps+1), gb_fractions)
plt.xlabel("Monte Carlo Steps")
plt.ylabel("Grain Boundary Fraction")
plt.title("Evolution of Grain Boundary Fraction")
plt.grid(True)
plt.show()

**Question 5**: How does the grain boundary fraction change over time? What does this indicate about the system's evolution? Can we interpret anything physically meaningful from the intermediate Monte Carlo steps (i.e., at 200, 400, 600, etc. steps)?

---

What happens when we change temperature? Go back into the code above (save your images and figures before you do), re-run the simulation at two additional temperatures, **100** and **1000 K**, and observe how the grain structure and grain boundary fraction evolve.

**Question 6**: How does temperature affect the final grain structure and the grain boundary fraction? Relate this to the concept of thermal fluctuations and energy minimization.

# Part 3: Thermodynamic Stabilization of Grain Boundaries Through Monte Carlo

In this portion of the assignment, you will explore a more realistic statistical mechanics model of nanocrystalline alloys, motivated by the thermodynamic stabilization framework developed by Schuh (https://www.science.org/doi/10.1126/science.1224737). Unlike the simpler models in the earlier parts, this simulation incorporates three key thermodynamic contributions that determine whether a fine-grained structure can be stabilized:


1.    **Grain Growth Energetics**

      Grain boundary energy drives coarsening: larger grains decrease the total boundary area and reduce the system’s energy. In our model, this is represented by a cost for unlike neighboring grain orientations.

2.    **Solute Segregation**

      Solute atoms have an energetic preference for occupying grain boundary sites. A negative segregation energy (ΔH_{seg}) lowers the system’s free energy by “pinning” grain boundaries and reducing their effective mobility.

3.    **Mixing Enthalpy**

      Solute and solvent atoms may energetically prefer mixing or demixing. A positive mixing enthalpy (ΔH_{mix}) promotes segregation of solute into certain regions (and sometimes phase separation), while a negative value favors a homogeneous solid solution.

Using Monte Carlo sweeps, our simulation alternates between grain growth moves (a lattice site adopting a neighbor’s orientation) and solute swap moves (solute and solvent atoms exchanging positions). Each attempted change is accepted or rejected based on the Metropolis criterion, ensuring that the system evolves toward thermodynamic equilibrium while retaining the effects of thermal fluctuations.



---

This first section of code contains all of our simulations parameters. Feel free to change Hseg_kJmol, Hmix_kJmol, and T as needed (positive to negative, increase, decrease values).

In [ ]:
from scipy import ndimage

# === Conversion Utility ===
def kJmol_to_J(H_kJmol):
    """Convert kJ/mol to J per entity (atom or bond)."""
    N_A = 6.02214076e23  # Avogadro's number
    return (H_kJmol * 1000.0) / N_A


# === Simulation Parameters (user: kJ/mol) ===
L = 60                   # Lattice size
n_orient = 8             # Number of grain orientations
X_B = 0.40               # Bulk solute concentration

gamma_gb_kJmol = 30.0   # GB energy per GB site (kJ/mol)
Hseg_kJmol = -50.0        # Segregation enthalpy (kJ/mol)
Hmix_kJmol = 50.0         # Mixing enthalpy per unlike neighbor pair at GB (kJ/mol)
T = 300                  # Temperature in K
k_B = 1.380649e-23       # Boltzmann constant (J/K)
rng = np.random.default_rng(2025)  # RNG seed

# === Convert to per-site or per-pair energies in Joules ===
gamma_gb = kJmol_to_J(gamma_gb_kJmol)
E_seg = kJmol_to_J(Hseg_kJmol)
J_mix = kJmol_to_J(Hmix_kJmol)

params = dict(
    gamma_gb=gamma_gb,
    E_seg=E_seg,
    J_mix=J_mix,
    k_B=k_B,
    T=T
)


This is the Python class responsible for generating the lattice.

In [ ]:
class PolycrystalLattice:
    def __init__(self, L, n_orient, X_B, rng):
        self.L = L
        self.n_orient = n_orient
        self.rng = rng
        self.grains = rng.integers(0, n_orient, size=(L, L))
        self.is_solute = np.zeros((L, L), dtype=bool)
        solute_indices = rng.choice(L*L, int(X_B*L*L), replace=False)
        self.is_solute.flat[solute_indices] = True

    def neighbors(self, i, j):
        L = self.L
        return [((i+1)%L, j), ((i-1)%L, j), (i, (j+1)%L), (i, (j-1)%L)]

    def is_boundary(self, i, j):
        s = self.grains[i, j]
        return any(self.grains[ni, nj] != s for ni, nj in self.neighbors(i, j))

    def site_energy(self, i, j, params):
        E = 0.0
        if self.is_boundary(i, j):
            E += params['gamma_gb']
            if self.is_solute[i, j]:
                E += params['E_seg']
            # Mixing enthalpy only between GB neighbors
            species = self.is_solute[i, j]
            for ni, nj in self.neighbors(i, j):
                if self.is_boundary(ni, nj) and self.is_solute[ni, nj] != species:
                    E += 0.5 * params['J_mix']  # Half to avoid overcounting
        return E

    def total_energy(self, params):
        E = 0.0
        for i in range(self.L):
            for j in range(self.L):
                E += self.site_energy(i, j, params)
        return E

    def grain_boundary_fraction(self):
        gb_sites = sum(self.is_boundary(i, j) for i in range(self.L) for j in range(self.L))
        return gb_sites / (self.L * self.L)

    def solute_gb_fraction(self):
        gb_solute = 0
        total_solute = np.sum(self.is_solute)
        for i in range(self.L):
            for j in range(self.L):
                if self.is_solute[i, j] and self.is_boundary(i, j):
                    gb_solute += 1
        return gb_solute / total_solute if total_solute > 0 else 0

    def average_grain_size(self):
        labels, n_grains = ndimage.label(self.grains)
        sizes = ndimage.sum(np.ones_like(self.grains), labels, range(1, n_grains+1))
        return np.mean(sizes) if n_grains > 0 else 0

    def plot_grains(self, title="Grains", ax=None):
        if ax is None:
            fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(self.grains, cmap='tab20', interpolation='nearest')
        ax.set_title(title)
        ax.axis('off')
        if ax is None:
            plt.show()

    def plot_solute(self, title="Solute Distribution", ax=None):
        if ax is None:
            fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(self.is_solute, cmap='coolwarm', interpolation='nearest')
        ax.set_title(title)
        ax.axis('off')
        if ax is None:
            plt.show()

    def plot_overlay(self, title="Grains + Solute Overlay", ax=None):
        if ax is None:
            fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(self.grains, cmap='tab20', interpolation='nearest', alpha=0.8)
        ax.imshow(np.where(self.is_solute, 1, np.nan), cmap='cool', interpolation='nearest', alpha=0.5)
        ax.set_title(title)
        ax.axis('off')
        if ax is None:
            plt.show()


This is the Python class that contains all the code for running the Monte Carlo simulation.

In [ ]:
class GrainBoundaryMC:
    def __init__(self, lattice, params, rng, move_probs=(0.5, 0.5)):
        self.lattice = lattice
        self.params = params
        self.rng = rng
        self.move_probs = move_probs  # (grain growth, solute swap)

    def mc_sweep(self):
        L = self.lattice.L
        for _ in range(L * L):
            move_type = self.rng.choice(['grain', 'solute'], p=self.move_probs)
            if move_type == 'grain':
                self.grain_growth_move()
            else:
                self.solute_swap_move()

    def grain_growth_move(self):
        i, j = self.rng.integers(0, self.lattice.L, size=2)
        s_old = self.lattice.grains[i, j]
        neighbors = self.lattice.neighbors(i, j)
        ni, nj = neighbors[self.rng.integers(0, 4)]
        s_new = self.lattice.grains[ni, nj]
        if s_new == s_old:
            return
        E_before = self.lattice.site_energy(i, j, self.params)
        self.lattice.grains[i, j] = s_new
        E_after = self.lattice.site_energy(i, j, self.params)
        delta_E = E_after - E_before
        if delta_E > 0 and self.rng.random() > np.exp(-delta_E / (self.params['k_B'] * self.params['T'])):
            self.lattice.grains[i, j] = s_old  # reject

    def solute_swap_move(self):
        L = self.lattice.L
        i1, j1 = self.rng.integers(0, L, size=2)
        i2, j2 = self.rng.integers(0, L, size=2)
        if self.lattice.is_solute[i1, j1] == self.lattice.is_solute[i2, j2]:
            return
        E_before = self.lattice.site_energy(i1, j1, self.params) + self.lattice.site_energy(i2, j2, self.params)
        self.lattice.is_solute[i1, j1], self.lattice.is_solute[i2, j2] = self.lattice.is_solute[i2, j2], self.lattice.is_solute[i1, j1]
        E_after = self.lattice.site_energy(i1, j1, self.params) + self.lattice.site_energy(i2, j2, self.params)
        delta_E = E_after - E_before
        if delta_E > 0 and self.rng.random() > np.exp(-delta_E / (self.params['k_B'] * self.params['T'])):
            # reject
            self.lattice.is_solute[i1, j1], self.lattice.is_solute[i2, j2] = self.lattice.is_solute[i2, j2], self.lattice.is_solute[i1, j1]


This code runs the simulation. Be prepared for it to take a few minutes.

In [ ]:
# === Initialize Lattice and Simulator ===
lattice = PolycrystalLattice(L, n_orient, X_B, rng)
mc_sim = GrainBoundaryMC(lattice, params, rng)

# === Data Tracking ===
n_sweeps = 1000
gb_fractions = []
solute_gb_fractions = []
avg_grain_sizes = []
snapshots = [0, 9, 99, 499, n_sweeps-1]

# === Initial Plots ===
fig, axes = plt.subplots(1, 3, figsize=(15,5))
lattice.plot_grains("Initial Grain Orientations", ax=axes[0])
lattice.plot_solute("Initial Solute Distribution", ax=axes[1])
lattice.plot_overlay("Initial Overlay", ax=axes[2])
plt.tight_layout()
plt.show()

# === Monte Carlo Sweeps ===
for sweep in range(n_sweeps):
    mc_sim.mc_sweep()
    gb_fractions.append(lattice.grain_boundary_fraction())
    solute_gb_fractions.append(lattice.solute_gb_fraction())
    avg_grain_sizes.append(lattice.average_grain_size())
    if sweep in snapshots:
        fig, axes = plt.subplots(1, 3, figsize=(15,5))
        lattice.plot_grains(f"Grain Orientations: Step {sweep+1}", ax=axes[0])
        lattice.plot_solute(f"Solute Distribution: Step {sweep+1}", ax=axes[1])
        lattice.plot_overlay(f"Overlay: Step {sweep+1}", ax=axes[2])
        plt.tight_layout()
        plt.show()

# === Final Plot ===
plt.figure(figsize=(8,5))
plt.plot(gb_fractions, label="Grain Boundary Fraction")
plt.plot(solute_gb_fractions, label="Solute @ GB Fraction")
plt.plot(np.array(avg_grain_sizes)/(L*L), label="Avg Grain Size (fraction)")
plt.xlabel("MC Sweep")
plt.ylabel("Fraction / Normalized Size")
plt.title("Microstructural Evolution")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

**Question 7**: What happens to the fraction of solute atoms located at grain boundaries as the simulation runs? How does this behavior depend on the value (sign and magnitude) of the segregation energy (ΔH_{seg})?


---


**Question 8**: How does changing the mixing enthalpy parameter (ΔH_{mix}) alter the equilibrium microstructure?


---


**Question 9**: How does raising or lowering the temperature (T) impact the balance between grain growth and solute segregation? Relate your observations to the role of thermal fluctuations in overcoming energy barriers.


---


**Question 10**: Under what conditions (values of ΔH_{seg}, ΔH_{mix}, T) did you observe the alloy maintaining a stable fine-grained structure?